<a href="https://colab.research.google.com/github/sku1978/sk-share-repo/blob/main/Spark/SparkDataFrame/SparkDataFrameNotebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq  > /dev/null 
!wget -q https://downloads.apache.org/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz > /dev/null 
!pip install -q findspark

In [ ]:
!mkdir /content/conf /content/lib
!wget -O /content/conf/log4j.properties https://raw.githubusercontent.com/sku1978/sk-share-repo/main/Spark/SparkDataFrame/conf/log4j.properties > /dev/null 2>&1
!mv /content/spark-3.1.1-bin-hadoop3.2/conf/spark-defaults.conf /content/spark-3.1.1-bin-hadoop3.2/conf/spark-defaults.conf.bk  > /dev/null 2>&1
!wget -O /content/spark-3.1.1-bin-hadoop3.2/conf/spark-defaults.conf https://raw.githubusercontent.com/sku1978/sk-share-repo/main/Spark/SparkDataFrame/conf/spark-defaults.conf  > /dev/null 2>&1
!wget -O /content/conf/spark.conf https://raw.githubusercontent.com/sku1978/sk-share-repo/main/Spark/SparkDataFrame/conf/spark.conf > /dev/null 2>&1

!wget -O /content/lib/logger.py https://raw.githubusercontent.com/sku1978/sk-share-repo/main/Spark/SparkDataFrame/lib/logger.py  > /dev/null 2>&1
!wget -O /content/lib/utils.py https://raw.githubusercontent.com/sku1978/sk-share-repo/main/Spark/SparkDataFrame/lib/utils.py  > /dev/null 2>&1

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

import findspark
findspark.init()
findspark.find()

'/content/spark-3.1.1-bin-hadoop3.2'

In [ ]:
#!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
#!unzip ngrok-stable-linux-amd64.zip
#get_ipython().system_raw('./ngrok http 4050 &')

In [ ]:
#!curl -s http://localhost:4040/api/tunnels

In [ ]:
def load_survey_df(spark, url):
   spark.sparkContext.addFile(url)

   survey_df=spark.read \
   .format("csv") \
   .option("header", "true") \
   .option("inferSchema", "true") \
   .load('file://'+SparkFiles.get("sample.csv"))

   return survey_df

def count_by_country(survey_df):
  count_df= survey_df.select("Age", "Gender", "Country", "State") \
                     .where("Age <= 40") \
                     .groupBy("Country") \
                     .count()
  return count_df

In [ ]:
from pyspark.sql import *
from pyspark import SparkConf, SparkFiles
from lib.logger import Log4J
from lib.utils import get_spark_app_config

conf=get_spark_app_config()

spark = SparkSession.builder\
        .config(conf=conf)\
        .getOrCreate()

logger = Log4J(spark)

logger.info("Staring Application")

url='https://raw.githubusercontent.com/sku1978/sk-share-repo/main/Spark/SparkDataFrame/data/sample.csv'

survey_df=load_survey_df(spark, url)
partitioned_survey_df=survey_df.repartition(2)

count_df=count_by_country(partitioned_survey_df)

logger.info(count_df.collect())

logger.info("Ending Application")

In [ ]:
!cat app-logs/sparklog.log

21/04/05 14:59:53 INFO Hello World: Staring Application
21/04/05 15:00:03 INFO Hello World: [[United Kingdom, 1], [Canada, 2], [United States, 4]]
21/04/05 15:00:03 INFO Hello World: Ending Application
